In [2]:
import pandas as pd
import re


In [3]:
zebraLogicBench = pd.read_parquet("hf://datasets/allenai/ZebraLogicBench/grid_mode/test-00000-of-00001.parquet")

In [4]:
zebraLogicBench[0:2]

,id,size,puzzle,solution,created_at
0,lgp-test-5x6-16,5*6,"There are 5 houses, numbered 1 to 5 from left ...","{'header': ['House', 'Name', 'Nationality', 'B...",2024-07-11T01:22:10.734298
1,lgp-test-4x4-27,4*4,"There are 4 houses, numbered 1 to 4 from left ...","{'header': ['House', 'Name', 'Occupation', 'Bo...",2024-07-11T01:22:10.732670


In [5]:
puzzle1 = zebraLogicBench['puzzle'][0]
print(puzzle1)

There are 5 houses, numbered 1 to 5 from left to right, as seen from across the street. Each house is occupied by a different person. Each house has a unique attribute for each of the following characteristics:
 - Each person has a unique name: `Peter`, `Alice`, `Bob`, `Eric`, `Arnold`
 - The people are of nationalities: `norwegian`, `german`, `dane`, `brit`, `swede`
 - People have unique favorite book genres: `fantasy`, `biography`, `romance`, `mystery`, `science fiction`
 - Everyone has something unique for lunch: `stir fry`, `grilled cheese`, `pizza`, `spaghetti`, `stew`
 - Each person has a favorite color: `red`, `green`, `blue`, `yellow`, `white`
 - The people keep unique animals: `bird`, `dog`, `cat`, `horse`, `fish`

## Clues:
1. The person who loves fantasy books is the Norwegian.
2. The cat lover and the person who loves biography books are next to each other.
3. The German is Bob.
4. The person who loves yellow is Bob.
5. The person whose favorite color is green is Peter.
6. 

In [6]:
# result = re.findall(r'There are (\d+) houses, numbered 1 to \d+ from left to right, as seen from across the street. Each house is occupied by a different person. Each house has a unique attribute for each of the following characteristics:', puzzle1)
result = re.findall(r'There are (\d+) houses, numbered 1 to \d+ from left to right', puzzle1)

if len(result) != 1:
    print("Invlaid count")

house_count = int(result[0])

house_count

5

In [7]:

# result = re.findall(r' - (.*?): (`(.*?)`)(?:, (`(.*?)`))*', puzzle1)

x = []
for i in range(0, house_count):
    x.append('`(.*?)`')

var_parser = ", ".join(x)

variables = re.findall(r' - (.*?): ' + var_parser, puzzle1)
variables


[('Each person has a unique name', 'Peter', 'Alice', 'Bob', 'Eric', 'Arnold'),
 ('The people are of nationalities',
  'norwegian',
  'german',
  'dane',
  'brit',
  'swede'),
 ('People have unique favorite book genres',
  'fantasy',
  'biography',
  'romance',
  'mystery',
  'science fiction'),
 ('Everyone has something unique for lunch',
  'stir fry',
  'grilled cheese',
  'pizza',
  'spaghetti',
  'stew'),
 ('Each person has a favorite color',
  'red',
  'green',
  'blue',
  'yellow',
  'white'),
 ('The people keep unique animals', 'bird', 'dog', 'cat', 'horse', 'fish')]

In [8]:
# 1. The person who loves fantasy books is the Norwegian.
clues = re.findall(r'\d+. (.*?)\.', puzzle1)


clues

['The person who loves fantasy books is the Norwegian',
 'The cat lover and the person who loves biography books are next to each other',
 'The German is Bob',
 'The person who loves yellow is Bob',
 'The person whose favorite color is green is Peter',
 'There is one house between the Dane and the person who is a pizza lover',
 'The person who loves blue is somewhere to the left of the Dane',
 'The person who loves eating grilled cheese is somewhere to the left of the Norwegian',
 'The person who loves the spaghetti eater is Peter',
 'The person who keeps horses is Alice',
 'The fish enthusiast is directly left of the person who loves science fiction books',
 'There is one house between the Norwegian and Arnold',
 'The person who loves romance books is the British person',
 'There are two houses between the Norwegian and Alice',
 'The bird keeper is the person whose favorite color is red',
 'The dog owner is directly left of the fish enthusiast',
 'The person who loves the stew is the 

In [24]:
def check_for_clue(var1, var2, regex, text):
    
    sa = regex.replace("%1", var1).replace("%2", var2)
    if re.search(sa, text, re.IGNORECASE):
        return True
    
    sb = regex.replace("%1", var2).replace("%2", var1)
    if re.search(sb, text, re.IGNORECASE):
        return True
    
    return False
    

def analyze_clue(vars, clue):
    
    if len(vars) == 2:
        if check_for_clue(vars[0], vars[1], "one house between(.*?)%1(.*?)%2", clue):
            return "oneBetween"
        if check_for_clue(vars[0], vars[1], "two houses between(.*?)%1(.*?)%2", clue):
            return "twoBetween"
        if check_for_clue(vars[0], vars[1], "%1(.*?)%2(.*?)next to each other", clue):
            return "nextTo"
        if check_for_clue(vars[0], vars[1], "%1(.*?)directly left of(.*?)%2", clue):
            return "dLeftOf"
        if check_for_clue(vars[0], vars[1], "%1(.*?)left of(.*?)%2", clue):
            return "leftOf"
        if check_for_clue(vars[0], vars[1], "%1(.*?)directly right of(.*?)%2", clue):
            return "dRightOf"
        if check_for_clue(vars[0], vars[1], "%1(.*?)right of(.*?)%2", clue):
            return "rightOf"
        if check_for_clue(vars[0], vars[1], "%1(.*?)is (the )?%2", clue):
        # if check_for_clue(vars[0], vars[1], "%1(.*?)is(.*?)%2", clue):
            return "equal"

    return None


clues2 = []
for c in clues:
    
    vars = []
    
    for group in variables:
        for var in group:
            if re.search(var, c, re.IGNORECASE):
                vars.append(var)
    
    func = analyze_clue(vars, c)
    
    clues2.append( (c, vars, func) )


for c in clues2:
    if c[2] is None:
        print(c)
    # display((c[2], c[0]))
    # print(c[2], c[0])
